In [1]:
import pandas as pd
from statsmodels.tsa.stattools import grangercausalitytests
import numpy as np

In [91]:
companies_with_cap = pd.read_csv('../data/companies_after_2005.csv', encoding='latin1')
svi = pd.read_csv('../data/agregated_svi.csv', encoding='latin1')
edgar = pd.read_csv('../data/edgar_all.csv', encoding='latin1')
weekly_market = pd.read_csv('../data/Weekly_market.csv', encoding='latin1')

#function to check wether there are equal number of weeks each year and companies 
#what should be in a dataset: company symbols, week number, week return,week volume, svi, edgar, cap size
#to see: returns from svi all, retturns from svi high, returns from svi low, returns from edgar all/high/all
#the same with volume (function)
#How to choose a lag

In [92]:
edgar.head()

,Unnamed: 0.1,Unnamed: 0,Name,Symbol,CIK,Count,20230603,20230602,20230601,20230531,...,20050110,20050109,20050108,20050107,20050106,20050105,20050104,20050103,20050102,20050101
0,0,0,Microsoft,MSFT,789019.0,3185,56921,271138,3231,5595,...,448,197,171,511,834,770,358,379,145,117
1,1,1,Apple,AAPL,320193.0,4860,66453,68420,5430,5051,...,166,87,62,79,146,196,285,93,54,15
2,2,2,NVIDIA,NVDA,1045810.0,2133,20629,25069,4397,3841,...,63,20,13,84,21,158,162,235,436,269
3,3,3,Amazon,AMZN,1018724.0,1392,21237,26889,3056,3833,...,126,58,26,92,177,141,207,69,51,29
4,4,4,Alphabet (Google),GOOG,1652044.0,1644,36914,61859,6703,5943,...,0,0,0,0,0,0,0,0,0,0


In [93]:
svi.head(5)

,Unnamed: 0.1,Unnamed: 0,symbol,date,svi,date_category,monthly_score,agregated_score
0,0,0,MSFT,2005-01-02,18,1,1741,31338
1,1,1,MSFT,2005-01-09,19,1,1741,33079
2,2,2,MSFT,2005-01-16,12,1,1741,20892
3,3,3,MSFT,2005-01-23,20,1,1741,34820
4,4,4,MSFT,2005-01-30,15,1,1741,26115


In [94]:
common_values = (
    set(weekly_market['TICKER'])
    .intersection(set(svi['symbol']))
    .intersection(set(companies_with_cap['Symbol']))
    .intersection(set(edgar['Symbol']))
)

# Filter each dataset to include only rows with common 'Symbol' values
filtered_weekly_market = weekly_market[weekly_market['TICKER'].isin(common_values)]
filtered_svi = svi[svi['symbol'].isin(common_values)]
filtered_companies = companies_with_cap[companies_with_cap['Symbol'].isin(common_values)]
filtered_edgar = edgar[edgar['Symbol'].isin(common_values)]

In [102]:
common_values

{'A',
 'AAPL',
 'ABEV',
 'ABT',
 'ACGL',
 'ACN',
 'ADBE',
 'ADI',
 'ADP',
 'ADSK',
 'AEP',
 'AFL',
 'AG',
 'AIG',
 'AIV',
 'ALC',
 'ALL',
 'AMAT',
 'AMD',
 'AME',
 'AMGN',
 'AMT',
 'AMWD',
 'AMX',
 'AMZN',
 'AON',
 'APH',
 'ASML',
 'AXP',
 'AZN',
 'AZO',
 'BA',
 'BAC',
 'BANR',
 'BB',
 'BBVA',
 'BCE',
 'BDX',
 'BHE',
 'BIIB',
 'BKNG',
 'BLK',
 'BMO',
 'BMY',
 'BNS',
 'BP',
 'BSX',
 'BUD',
 'BUSE',
 'C',
 'CAL',
 'CAT',
 'CBRL',
 'CCI',
 'CDNS',
 'CGAU',
 'CHTR',
 'CI',
 'CL',
 'CM',
 'CMCO',
 'CMCSA',
 'CME',
 'CMI',
 'CNC',
 'CNI',
 'COF',
 'COP',
 'COST',
 'CP',
 'CPRT',
 'CPRX',
 'CQP',
 'CRM',
 'CSCO',
 'CSGP',
 'CSGS',
 'CSX',
 'CTAS',
 'CTS',
 'CTSH',
 'CVS',
 'CVX',
 'CXW',
 'D',
 'DBD',
 'DE',
 'DELL',
 'DEO',
 'DG',
 'DHI',
 'DHR',
 'DIS',
 'DLR',
 'DLTR',
 'DNN',
 'DVAX',
 'E',
 'EA',
 'ECL',
 'EFX',
 'EL',
 'EMR',
 'ENB',
 'EOG',
 'EPD',
 'EQIX',
 'EW',
 'EXC',
 'EXR',
 'EXTR',
 'F',
 'FAST',
 'FBNC',
 'FCF',
 'FCX',
 'FDX',
 'FERG',
 'FI',
 'FICO',
 'FIS',
 'FNLC',
 'FWRG',

In [96]:
filtered_weekly_market.shape

(325376, 8)

In [ ]:
filtered_svi.shape

In [99]:
# Extract the year from 'week_index'
filtered_weekly_market['year'] = filtered_weekly_market['week_index'].str.split('-').str[0].astype(int)

# Group by 'year' and calculate the count for each group
weeks_per_year_df2 = filtered_weekly_market.groupby('year').size()

# Print the counts
print("Weeks per year in df2:")
print(weeks_per_year_df2)

Weeks per year in df2:
year
2005    14779
2006    15000
2007    15585
2008    15592
2009    15537
2010    15546
2011    15748
2012    16192
2013    16343
2014    16435
2015    16556
2016    16367
2017    16432
2018    16852
2019    16953
2020    17099
2021    16919
2022    16985
2023    17027
dtype: int64


C:\Users\Варвара\AppData\Local\Temp\ipykernel_8116\1404412366.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_weekly_market['year'] = filtered_weekly_market['week_index'].str.split('-').str[0].astype(int)


In [100]:
filtered_svi['date'] = pd.to_datetime(filtered_svi['date'])

# Extract the year from the date
filtered_svi['year'] = filtered_svi['date'].dt.year

# Group by 'year' and calculate the count for each group
weeks_per_year = filtered_svi.groupby('year').size()

# Print the counts
print("Weeks per year:")
print(weeks_per_year)

Weeks per year:
year
2005    17056
2006    17384
2007    17056
2008    17056
2009    17056
2010    17056
2011    17056
2012    17384
2013    17056
2014    17056
2015    17056
2016    17056
2017    17384
2018    17056
2019    17056
2020    17056
2021    17056
2022    17056
2023    17384
dtype: int64


C:\Users\Варвара\AppData\Local\Temp\ipykernel_8116\1774035501.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_svi['date'] = pd.to_datetime(filtered_svi['date'])
C:\Users\Варвара\AppData\Local\Temp\ipykernel_8116\1774035501.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_svi['year'] = filtered_svi['date'].dt.year


In [98]:
filtered_svi['date'] = pd.to_datetime(filtered_svi['date'])
filtered_svi['week_number'] = filtered_svi['date'].dt.isocalendar().week
filtered_svi.head(55)

C:\Users\Варвара\AppData\Local\Temp\ipykernel_8116\676306728.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_svi['date'] = pd.to_datetime(filtered_svi['date'])
C:\Users\Варвара\AppData\Local\Temp\ipykernel_8116\676306728.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_svi['week_number'] = filtered_svi['date'].dt.isocalendar().week


,Unnamed: 0.1,Unnamed: 0,symbol,date,svi,date_category,monthly_score,agregated_score,week_number
0,0,0,MSFT,2005-01-02,18,1,1741,31338,53
1,1,1,MSFT,2005-01-09,19,1,1741,33079,1
2,2,2,MSFT,2005-01-16,12,1,1741,20892,2
3,3,3,MSFT,2005-01-23,20,1,1741,34820,3
4,4,4,MSFT,2005-01-30,15,1,1741,26115,4
5,5,5,MSFT,2005-02-06,16,1,1741,27856,5
6,6,6,MSFT,2005-02-13,20,1,1741,34820,6
7,7,7,MSFT,2005-02-20,16,1,1741,27856,7
8,8,8,MSFT,2005-02-27,17,1,1741,29597,8
9,9,9,MSFT,2005-03-06,15,1,1741,26115,9


In [88]:
ff = filtered_weekly_market[filtered_weekly_market['TICKER'] == 'MSFT']
ff.head(53)

,TICKER,COMNAM,week_index,week_cls_prc,week_vol,weekly_return,year
998,MSFT,MICROSOFT CORP,2005-01,26.67,398924026.0,NaN,2005
999,MSFT,MICROSOFT CORP,2005-02,26.12,379712121.0,-0.020622,2005
1000,MSFT,MICROSOFT CORP,2005-03,25.65,266617523.0,-0.017994,2005
1001,MSFT,MICROSOFT CORP,2005-04,26.18,409844550.0,0.020663,2005
1002,MSFT,MICROSOFT CORP,2005-05,26.32,347830186.0,0.005348,2005
1003,MSFT,MICROSOFT CORP,2005-06,25.97,360145305.0,-0.013298,2005
1004,MSFT,MICROSOFT CORP,2005-07,25.48,348529709.0,-0.018868,2005
1005,MSFT,MICROSOFT CORP,2005-08,25.25,329158361.0,-0.009027,2005
1006,MSFT,MICROSOFT CORP,2005-09,25.17,322997251.0,-0.003168,2005
1007,MSFT,MICROSOFT CORP,2005-10,25.09,318787290.0,-0.003178,2005


In [ ]:
#test function 
def grangertest(X, Y, lag_num, verbose=True):
    """
    Perform the Granger Causality Test on two time series.

    Parameters:
    - X: pandas Series or DataFrame
    - Y: pandas Series or DataFrame
    - lag_num: int, the number of lags to consider in the test
    - verbose: bool, whether to print the test results (default is True)

    Returns:
    - None
    """
    # Check if input data is a DataFrame, if not, convert to DataFrame
    if not isinstance(X, pd.DataFrame):
        X = pd.DataFrame(X, columns=['X'])
    if not isinstance(Y, pd.DataFrame):
        Y = pd.DataFrame(Y, columns=['Y'])

    # Concatenate X and Y for the Granger Causality Test
    data = pd.concat([X, Y], axis=1)

    # Perform Granger Causality Test
    test_result = grangercausalitytests(data, lag_num, verbose=verbose)

    # Extract the p-values from the test results
    p_values = [test_result[i+1][0]['ssr_ftest'][1] for i in range(lag_num)]

    # Print the test statistics and conclusion
    if verbose:
        print(f"Granger Causality Test Results:")
        for i in range(1, lag_num + 1):
            print(f"\nLag {i} - p-value: {p_values[i-1]}")
            if p_values[i-1] < 0.05:
                print(f"Conclusion: There is Granger causality at lag {i}")
            else:
                print(f"Conclusion: There is no Granger causality at lag {i}")
